### Create Technical Threshold Rasters
In this notebook we create three technical threshold rasters (1) Slope threshold raster (2) Population threshold raster (3) Flood risk threshold raster. These rasters will be masked from the Technical Inclusion Area Layer to generate the SL1 Inclusion Area Layer. This raster generates 'Technical Exclusion' layers simultaneously for both solar and wind resources, as the slope component is the only one that differs based on technology. 

In [1]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

#### Define Helpful Functions

In [2]:
def get_datum_transformation(source_spatial_reference, target_spatial_reference):
    '''
    This function retrieves the most appropriate datum transformation for converting spatial data
    from the source spatial reference to the target spatial reference.
    '''
    
    transformations = arcpy.ListTransformations(source_spatial_reference, target_spatial_reference)
    if transformations:
        return transformations[0]  # Use the first transformation found
    else:
        return None  # No appropriate transformation found

In [3]:
def add_occupancy_field(feature_class):
    '''
    Adds an 'occupancy' field to a feature class if it doesn't already exist and sets the value to 1 for all records.

    This function checks if the 'occupancy' field already exists in the specified feature class.
    If the field doesn't exist, it adds the 'occupancy' field and sets the value to 1 for all records.
    '''
    # Check if the 'occupancy' field already exists
    fields = arcpy.ListFields(feature_class, "occupancy")

    if not fields:
         # Add 'occupancy' field to the feature class if it doesn't exist
        arcpy.management.AddField(feature_class, "occupancy", "SHORT")

        # Set the 'occupancy' field value to 1 for all records
        with arcpy.da.UpdateCursor(feature_class, "occupancy") as cursor:
            for row in cursor:
                row[0] = 1
                cursor.updateRow(row)
        print(f"'occupancy' field added and set to 1 in {os.path.basename(feature_class)}")
    else:
        print(f"'occupancy' field already exists in {os.path.basename(feature_class)}")

In [5]:
def convert_to_one_bit(input_raster, output_raster):
    '''
    This function alters the bit depth of a raster. It is designed for the output of a Reclassify tool. 
    '''
    
    try:
        # Create a raster object from the input path
        input_raster_obj = arcpy.sa.Raster(input_raster)

        # Reclassify the input raster to binary (one-bit) format
        binary_raster = arcpy.sa.Con(input_raster_obj > 0, 1)

        # Set the pixel type of the output raster to 1_BIT
        arcpy.management.CopyRaster(binary_raster, output_raster, pixel_type="1_BIT")

        print(f"Conversion to one-bit raster completed successfully. Result saved to {output_raster}")

        # Clean up workspace
        try: 
            arcpy.Delete_management(binary_raster)
            arcpy.Delete_management(input_raster_obj)
        except Exception as e:
            print(f"An error occurred while cleaning up: {str(e)}")

    except arcpy.ExecuteError:
        print(arcpy.GetMessages())
    except Exception as e:
        print(f"An error occurred: {str(e)}")

#### Get Slope Threshold Raster

In [6]:
# Set input and output folder
mainInputFolder = "C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data"  # enter path to input folder

In [7]:
# Set path to wind workspace
wind_workspace = os.path.join(mainInputFolder, "technicalExclusionRasters\\Wind")

if not os.path.exists(wind_workspace):
    os.makedir(wind_workspace)

# Set path to solar workspace
solar_workspace = os.path.join(mainInputFolder, "technicalExclusionRasters\\Solar")

if not os.path.exists(solar_workspace):
    os.makedir(solar_workspace)

In [8]:
# Set path to input DEM
input_dem = os.path.join(mainInputFolder, "SRTM_90m.tif")

# Set path to output projected DEM,resample DEM, and slope raster
projected_dem = os.path.join(mainInputFolder, os.path.splitext(os.path.basename(input_dem))[0] + "_proj.tif")
resampled_dem = os.path.join(mainInputFolder, "SRTM_250m_proj.tif")
slope_raster =  os.path.join(mainInputFolder, "SRTM_250m_slope.tif")

In [ ]:
# get the input spatial reference
input_sr = arcpy.Describe(input_dem).spatialReference
input_sr

In [10]:
# Specify the output spatial reference
output_sr = arcpy.SpatialReference(102039)

# Specify the target cell size
cell_size = "250"

In [ ]:
# Define datum transform
datum_transform = get_datum_transformation(input_sr, output_sr)
datum_transform

In [ ]:
# Set overwrite mode to true
arcpy.env.overwriteOutput = True

try:
    # Project the DEM
    arcpy.ProjectRaster_management(in_raster = input_dem,
                                   out_raster = projected_dem,
                                   out_coor_system = output_sr,
                                   resampling_type = 'BILINEAR',
                                   geographic_transform = datum_transform)
    print("DEM reprojected successfully")

    # Resample the projected DEM to the target cell size
    arcpy.Resample_management(projected_dem, resampled_dem, cell_size, "BILINEAR")
    print("DEM resampled successfully")

    # Calculate the slope of the resampled DEM
    arcpy.Slope_3d(resampled_dem, slope_raster, "PERCENT_RISE")
    print("Slope calculation completed successfully!")

except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except:
    print(Exception)

In [ ]:
# Reclassified slope raster (threshold raster) for wind
wind_slope_threshold = os.path.join(wind_workspace, os.path.splitext(os.path.basename(slope_raster))[0]+ "_wind.tif")
wind_slope_threshold

In [ ]:
# Set file path for bit depth conversion
wind_slope_binary = os.path.join(wind_workspace, os.path.splitext(os.path.basename(wind_slope_threshold))[0]+ "_binary.tif")
wind_slope_binary

In [11]:
# Define the remap table
remap_table_wind = arcpy.sa.RemapRange([
    [0, 20, 0],
    [20, 2000, 1],
])

In [12]:
# Reclassify the slope raster using the remap table
reclassified_wind_slope = arcpy.sa.Reclassify(slope_raster, "VALUE", remap_table_wind)

# Save the reclassified raster to the output file path
reclassified_wind_slope.save(wind_slope_threshold)

In [ ]:
# Reclassified slope raster (threshold raster) for wind
solar_slope_threshold = os.path.join(solar_workspace, os.path.splitext(os.path.basename(slope_raster))[0]+ "_solar.tif")
solar_slope_threshold

In [ ]:
# Set file path for bit depth conversion
solar_slope_binary = os.path.join(solar_workspace, os.path.splitext(os.path.basename(solar_slope_threshold))[0]+ "_binary.tif")
solar_slope_binary

In [14]:
# Define the remap table
remap_table_solar = arcpy.sa.RemapRange([
    [0, 10, 0],
    [10, 2000, 1],
])

In [15]:
# Reclassify the slope raster using the remap table
reclassified_solar_slope = arcpy.sa.Reclassify(slope_raster, "VALUE", remap_table_solar)

# Save the reclassified raster to the output file path
reclassified_solar_slope.save(solar_slope_threshold)

In [ ]:
# Convert to binary raster
convert_to_one_bit(solar_slope_threshold, solar_slope_binary) # This does not work as intended, bit depth reduced from 16 bit to 8 bit 
convert_to_one_bit(wind_slope_threshold, wind_slope_binary) # This does not work as intended, bit depth reduced from 16 bit to 8 bit

In [10]:
# set environmental raster settings to the reclassified slope raster
arcpy.env.snapRaster = slope_raster
arcpy.env.extent = slope_raster
arcpy.env.cellSize = slope_raster

#### Get Population Threshold Raster

In [16]:
# Set path to input LANDSCAN raster
landscan_usa = os.path.join(mainInputFolder,"landscan-usa-2020-conus-day.tif")

In [36]:
# set paths to projected, resampled, and reclassified landscan raster
landscan_projected = os.path.join(mainInputFolder, "landscan_conus_2020_projected.tif")
landscan_resampled = os.path.join(mainInputFolder, "landscan_conus_2020_250m.tif")
landscan_threshold = os.path.join(mainInputFolder, "technicalExclusionRasters\\landscan_conus_2020_threshold_100.tif")


In [ ]:
# Get spatial reference of landscan raster
landscan_sr = arcpy.Describe(landscan_usa).spatialReference
landscan_sr

In [ ]:
# Get datum transform
landscan_datum_transform = get_datum_transformation(landscan_sr, output_sr)
landscan_datum_transform

In [ ]:
try:
    # Project the LANDSCAN raster
    arcpy.ProjectRaster_management(in_raster = landscan_usa,
                                   out_raster = landscan_projected,
                                   out_coor_system = output_sr,
                                   resampling_type = 'BILINEAR',
                                   geographic_transform = datum_transform)
    print("LANDSCAN reprojected successfully")

    # Resample the projected LANDSCAN raster to the target cell size
    arcpy.Resample_management(landscan_projected, landscan_resampled, cell_size, "BILINEAR")
    print("LANDSCAN resampled successfully")
    
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except:
    print(Exception)

In [25]:
# Define the remap table
landscan_remap = arcpy.sa.RemapRange([
[0,0.81, 0],
[0.81, 15500, 1] # remap values correspond to 100 people/sq km (via landscan documentation)
])

In [27]:
# Reclassify the LANDSCAN raster using the remap table
landscan_reclassified = arcpy.sa.Reclassify(landscan_resampled, "VALUE", landscan_remap)

# Save the reclassified raster to the output file path
landscan_reclassified.save(landscan_threshold)

In [ ]:
# Set path to binary raster 
landscan_binary = os.path.join(mainInputFolder, "technicalExclusionRasters\\landscan_conus_2020_threshold_100_binary.tif")
landscan_binary

In [ ]:
# convert to binary raster
convert_to_one_bit(landscan_threshold, landscan_binary) #  This appears to not work here, bit depth reduced from 16 bit to 8 bit

#### Get Flood Threshold Raster

In [11]:
# Set path to flood hazard layer
flood_hazard = os.path.join(mainInputFolder, "floodHazard.gdb\\FEMA_flood_hazard_zones")


In [12]:
# Set path to projected flood hazard layer and raster layer
flood_hazard_projected = os.path.join(mainInputFolder, "floodHazard.gdb\\FEMA_flood_hazard_zones_projected")
flood_hazard_raster = os.path.join(mainInputFolder, "technicalExclusionRasters\\FEMA_flood_hazard_zones.tif")

In [ ]:
# Get flood hazard projection
flood_hazard_sr = arcpy.Describe(flood_hazard).spatialReference
flood_hazard_sr

In [14]:
# get datum transformation
flood_hazard_datum_transformation = get_datum_transformation(flood_hazard_sr, output_sr)
flood_hazard_datum_transformation

In [ ]:
# Project the flood hazard layer
try:
    arcpy.Project_management(in_dataset = flood_hazard,
                                   out_dataset = flood_hazard_projected,
                                   out_coor_system = output_sr,
                                   transform_method = flood_hazard_datum_transformation)
    print("flood hazard reprojected successfully")
 
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except:
    print(Exception)

In [ ]:
# add occupancy field to the flood hazard layer and convert to raster
add_occupancy_field(flood_hazard_projected)
arcpy.conversion.FeatureToRaster(in_features = flood_hazard_projected,
                                 field = "occupancy",
                                 out_raster = flood_hazard_raster)

#### Mosaic Technical Threshold Rasters

In [87]:
# Create a list of rasters to mosaic
wind_rasters = [flood_hazard_raster, landscan_binary, wind_slope_binary]
solar_rasters = [flood_hazard_raster, landscan_binary, solar_slope_binary]

In [88]:
# Set file path to save mosaic outputs
mosaic_workspace = os.path.join(mainInputFolder, "technicalExclusionRasters\\Mosaic")
mosaic_name_wind = "technicalExclusionMosaic_Wind.tif"
mosaic_name_solar = "technicalExclusionMosaic_Solar.tif"

In [ ]:
# Perform the mosaic using MosaicToNewRaster_management function
arcpy.MosaicToNewRaster_management(solar_rasters, mosaic_workspace, mosaic_name_solar, pixel_type="1_BIT", number_of_bands=1)
arcpy.MosaicToNewRaster_management(wind_rasters, mosaic_workspace, mosaic_name_wind, pixel_type="1_BIT", number_of_bands=1)